<a href="https://colab.research.google.com/github/mehmetsar/Analiz/blob/main/Writings_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install this package to use Colab's GPU for training
!apt install --allow-change-held-packages libcudnn8=8.4.1.50-1+cuda11.6

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be upgraded:
  libcudnn8
1 upgraded, 0 newly installed, 1 to remove and 18 not upgraded.
Need to get 420 MB of archives.
After this operation, 3,369 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libcudnn8 8.4.1.50-1+cuda11.6 [420 MB]
Fetched 420 MB in 8s (53.5 MB/s)
(Reading database ... 155676 files and directories currently installed.)
Removing libcudnn8-dev (8.0.5.39-1+cuda11.1) ...
(Reading database ... 155654 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.4.1.50-1+cuda11.6_amd64.deb ...
Unpacking libcudnn8 (8.4.1.50-1+c

In [1]:
# for loading/processing the images  
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 

# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model

# clustering and dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle


In [3]:
#Loading data

path = r"/content/sample_data/Clusters"
# change the working directory to the path where the images are located
os.chdir(path)

# this list holds all the image filename
writings = []

# creates a ScandirIterator aliased as files
with os.scandir(path) as files:
  # loops through each file in the directory
    for file in files:
        if file.name.endswith('.png'):
          # adds only the image files to the flowers list
            writings.append(file.name)
    

In [4]:
print(writings[:10])

['g06-011c-s00-01.png', 'g06-011e-s02-00.png', 'g06-011l-s01-02.png', 'g06-011m-s05-01.png', 'g06-011j-s00-00.png', 'g06-011e-s05-02.png', 'g06-011i-s05-02.png', 'g06-031n-s04-07.png', 'g06-011b-s01-00.png', 'g06-011n-s05-00.png']


In [6]:
# load the image as a 224x224 array
img = load_img(writings[0], target_size=(224,224))
# convert from 'PIL.Image.Image' to numpy array
img = np.array(img)

print(img.shape)
(224, 224, 3)


(224, 224, 3)


(224, 224, 3)

In [8]:
reshaped_img = img.reshape(1,224,224,3)
print(reshaped_img.shape)
x = preprocess_input(reshaped_img)



(1, 224, 224, 3)


In [9]:
# load model
model = VGG16()
# remove the output layer
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

553476096/553467096 [==============================] - 5s 0us/step


In [10]:
features = model.predict(image)
print(features.shape)

NameError: ignored

In [11]:
model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

def extract_features(file, model):
    # load the image as a 224x224 array
    img = load_img(file, target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    return features

In [15]:
data = {}
p = r"/content/sample_data/"

# lop through each image in the dataset
for writing in writings:
    # try to extract the features and update the dictionary
    try:
        feat = extract_features(writing,model)
        data[writing] = feat
    # if something fails, save the extracted features as a pickle file (optional)
    except:
        with open(p,'wb') as file:
            pickle.dump(data,file)
          
 
# get a list of the filenames
filenames = np.array(list(data.keys()))

# get a list of just the features
feat = np.array(list(data.values()))
feat.shape




(210, 4096)

In [16]:
pca = PCA(n_components=100, random_state=22)
pca.fit(feat)
x = pca.transform(feat)

In [17]:
kmeans = KMeans(n_clusters=3,random_state=22)
kmeans.fit(x)

KMeans(n_clusters=3, random_state=22)

In [19]:
kmeans.labels_

array([0, 0, 2, 2, 2, 1, 1, 2, 1, 2, 0, 0, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2,
       2, 1, 1, 0, 2, 0, 2, 1, 0, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1,
       0, 0, 2, 2, 2, 2, 2, 1, 1, 0, 1, 2, 2, 2, 1, 2, 2, 2, 2, 1, 1, 2,
       1, 2, 2, 1, 1, 2, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 2, 1, 2, 1, 2,
       1, 0, 0, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 1, 1, 2, 0, 2, 1, 2, 2, 2,
       1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2, 1, 0, 2, 1,
       1, 0, 2, 2, 0, 2, 1, 2, 1, 1, 1, 0, 2, 1, 1, 0, 1, 0, 2, 0, 0, 2,
       0, 1, 2, 2, 2, 0, 1, 1, 1, 1, 0, 2, 2, 1, 1, 0, 0, 1, 1, 2, 0, 2,
       1, 1, 0, 2, 2, 0, 2, 1, 2, 2, 1, 2, 0, 1, 1, 2, 0, 1, 1, 1, 0, 1,
       0, 1, 2, 2, 0, 2, 2, 0, 2, 2, 1, 1, 0, 2, 0, 2, 2, 1, 1, 1, 0, 2,
       1, 2, 2, 0, 2, 0, 0, 2, 2, 2, 1, 2, 2, 0, 2, 0, 2, 0, 1, 1, 2, 1,
       1, 0, 0, 2, 2, 0, 1, 2, 2, 1, 0, 2, 2, 2, 2, 0, 0, 1, 0, 0, 2, 0,
       1, 2, 1, 0, 2, 0, 1, 0, 2, 0, 1, 1, 2, 0, 1, 0], dtype=int32)

In [20]:
# holds the cluster id and the images { id: [images] }
groups = {}
for file, cluster in zip(filenames,kmeans.labels_):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(file)
    else:
        groups[cluster].append(file)

In [21]:
groups[0]

['g06-011c-s00-01.png',
 'g06-011e-s02-00.png',
 'g06-011m-s03-01.png',
 'g06-011i-s02-00.png',
 'g06-011l-s03-00.png',
 'g06-031o-s03-00.png',
 'g06-011n-s01-03.png',
 'g06-011p-s03-00.png',
 'g06-011j-s04-00.png',
 'g06-011m-s04-01.png',
 'g06-011m-s02-01.png',
 'g06-011g-s01-00.png',
 'g06-011l-s01-00.png',
 'g06-011b-s05-00.png',
 'g06-011l-s04-00.png',
 'g06-031o-s03-02.png',
 'g06-011j-s01-02.png',
 'g06-011e-s05-00.png',
 'g06-011c-s02-01.png',
 'g06-031j-s04-07.png',
 'g06-011n-s04-00.png',
 'g06-011n-s04-02.png',
 'g06-011h-s02-02.png',
 'g06-011k-s01-02.png',
 'g06-031n-s03-02.png',
 'g06-031o-s04-00.png',
 'g06-011i-s04-00.png',
 'g06-011j-s03-00.png',
 'g06-011p-s01-00.png',
 'g06-011c-s04-01.png',
 'g06-011f-s05-00.png',
 'g06-011f-s03-01.png',
 'g06-011o-s00-01.png',
 'g06-011o-s03-01.png',
 'g06-011i-s03-00.png',
 'g06-011f-s02-01.png',
 'g06-011k-s05-04.png',
 'g06-011e-s03-00.png',
 'g06-011l-s05-00.png',
 'g06-011h-s03-01.png',
 'g06-011l-s02-00.png',
 'g06-031j-s02-0

In [22]:
import pandas as pd

ValueError: ignored

In [30]:
for i in range(len(groups)):
  print(len(groups[i]))

64
99
117


In [32]:
for i in range(117-64):
  groups[0].append('nan')

In [33]:
for i in range(117-99):
  groups[1].append('nan')

In [34]:
for i in range(len(groups)):
  print(len(groups[i]))

117
117
117


In [35]:
pd.DataFrame({'group1':groups[0],'group2':groups[1],'group3':groups[2]}).to_csv('clusters.csv')

In [ ]:
kmeans.predict(
    
)